# Platform Readiness Validation

**Phase**: Setup (00)  
**Objective**: Validate all platform infrastructure components before notebook execution  
**Time**: 3-5 minutes  
**Status**: Ready

## Overview

This notebook performs comprehensive validation of all platform components required for successful execution of the 32 notebooks in the Self-Healing Platform.

### What This Notebook Validates

1. **Basic Environment** (6 checks)
2. **Platform Infrastructure** (12 checks)
3. **OpenShift Components** (10 checks)
4. **Network Connectivity** (7 checks)

**Reference**: ADR-029 - Infrastructure Validation Notebook for User Readiness

In [ ]:
import sys
import os

# Add utils directory to path - works in both ValidationJobs and Workbench
possible_utils_paths = [
    '/workspace/repo/notebooks/utils',  # ValidationJob environment
    '/opt/app-root/src/openshift-aiops-platform/notebooks/utils',  # Workbench (preloaded repo)
    os.path.join(os.path.dirname(os.path.abspath(__file__) if '__file__' in dir() else os.getcwd()), '..', 'utils'),  # Relative path
]

utils_path_found = False
for path in possible_utils_paths:
    if os.path.exists(path):
        sys.path.insert(0, path)
        utils_path_found = True
        print(f"✅ Added utils path: {path}")
        break

if not utils_path_found:
    print("⚠️ Warning: Could not find utils directory, using current directory")
    sys.path.insert(0, os.path.dirname(os.getcwd()))

# Import validation helpers
from validation_helpers import (
    validate_python_version,
    validate_pytorch_installation,
    validate_gpu_availability,
    validate_storage_volumes,
    validate_coordination_engine,
    validate_coordination_engine_metrics,
    validate_coordination_engine_kserve_proxy,
    validate_model_serving,
    validate_object_storage,
    validate_mcp_server,
    validate_prometheus,
    validate_argocd_applications,
    validate_tekton_pipelines,
    validate_external_secrets,
    generate_validation_report,
    print_validation_report
)
print("✅ Validation functions imported successfully")

In [ ]:
print("=" * 80)
print("📦 VALIDATING BASIC ENVIRONMENT")
print("=" * 80)
print()
basic_checks = []
print("⏳ Checking Python version...")
basic_checks.append(validate_python_version())
print("⏳ Checking PyTorch...")
basic_checks.append(validate_pytorch_installation())
print("⏳ Checking GPU...")
basic_checks.append(validate_gpu_availability())
print("⏳ Checking storage...")
basic_checks.extend(validate_storage_volumes())
passed = sum(1 for c in basic_checks if c['status'] == 'PASSED')
print(f"\
📊 Basic Environment: {passed}/{len(basic_checks)} passed")


In [ ]:
print("=" * 80)
print("🏗️  VALIDATING PLATFORM INFRASTRUCTURE")
print("=" * 80)
platform_checks = []
print("⏳ Coordination Engine...")
platform_checks.append(validate_coordination_engine())
platform_checks.append(validate_coordination_engine_metrics())
platform_checks.append(validate_coordination_engine_kserve_proxy())
print("⏳ Model Serving...")
platform_checks.extend(validate_model_serving())
print("⏳ Object Storage...")
platform_checks.extend(validate_object_storage())
print("⏳ MCP Server...")
platform_checks.append(validate_mcp_server())
print("⏳ Prometheus...")
platform_checks.append(validate_prometheus())
passed = sum(1 for c in platform_checks if c['status'] == 'PASSED')
print(f"📊 Platform Infrastructure: {passed}/{len(platform_checks)} passed")


In [ ]:
print("\
=" * 80)
print("☸️  VALIDATING OPENSHIFT COMPONENTS")
print("=" * 80)
openshift_checks = []
print("⏳ ArgoCD...")
openshift_checks.append(validate_argocd_applications())
print("⏳ Tekton...")
openshift_checks.extend(validate_tekton_pipelines())
print("⏳ External Secrets...")
openshift_checks.extend(validate_external_secrets())
passed = sum(1 for c in openshift_checks if c['status'] == 'PASSED')
print(f"\
📊 OpenShift Components: {passed}/{len(openshift_checks)} passed")


In [ ]:
all_checks = basic_checks + platform_checks + openshift_checks
report = generate_validation_report(all_checks)
print(f"✅ Report saved to: /opt/app-root/src/.config/validation-report.json")


In [ ]:
# Print validation report
print_validation_report(report)

# Exit with appropriate code based on validation status
import sys
status = report["validation_status"]
if status == "FAILED":
    print("\n⚠️  Exiting with code 1 due to critical validation failures")
    sys.exit(1)
elif status == "DEGRADED":
    print("\n⚠️  Exiting with code 0 (warnings present but not blocking)")
    sys.exit(0)
else:
    print("\n✅ Exiting with code 0 (all validations passed)")
    sys.exit(0)